# Extracting on-chain data from a list of Ethereum public addresses

In this notebook, we leverage [Etherscan API](https://docs.etherscan.io/api-endpoints/accounts) to extract the following information from a list of public addresses (retrieved from the file defined on `ADRESS_LIST`):

1. Current ETH balance
2. Token transfers and swaps
3. Current tokens balance

💡 **Please add your [Etherscan API](https://etherscan.io/myapikey?msg=apikeyremove) info and the public address list to the `.env` file.**

<br>
<br>

## 0. Setup

<br>
<br>

### Installing dependencies

In [1]:
import os
import requests
import numpy as np
import pandas as pd
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import urljoin

In [2]:
load_dotenv()
env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)
 
ETHERSCAN_API = os.getenv("ETHERSCAN_API")
ETHERESCAN_ENDPOINT = os.getenv("ETHERESCAN_ENDPOINT")
INFURA_URL = os.getenv("INFURA_URL")
ADDRESS_LIST = os.getenv("ADDRESS_LIST")
OUTPUT_DIR = os.getenv("OUTPUT_DIR")

if not (bool(ETHERESCAN_ENDPOINT) and bool(ETHERSCAN_API) and bool(ADDRESS_LIST) and bool(INFURA_URL) and bool(OUTPUT_DIR)):
    raise Exception('Please add config to .env file')

<br>

### Defining util methods 

In [3]:
def send_request(query, less_log=False) -> dict:
    
    
    url = urljoin(ETHERESCAN_ENDPOINT, query)
    
    if less_log:
        print('Sending request to: {}'.format(url))
    
    try:
        r = requests.get(url)
        
    except requests.exceptions.HTTPError  as e:
        raise Exception('Error querying to {0}: {1}'.format(ETHERESCAN_ENDPOINT, e.response.text))
    
    if r.status_code == 200:
        return r.json()
    else:
        raise Exception('Query failed - return code:{}.'.format(r.status_code))

In [4]:
def wei_to_eth(num) -> float:
    return num / 1000000000000000000 

In [5]:
def handle_KeyError(response, key) -> str:    
    try:
        return response[key]
    except KeyError as e:
         raise Exception('Response cannot be parsed.'.format(e))

In [6]:
def open_file(filename) -> list:
    try:
        with open(filename,'r') as f:
            return f.readlines()
    except IOError as e:
         raise Exception('Could not open {0}: {1}'.format(filename, e))

In [7]:
def get_addresses_str(address_list) -> str:
    return ','.join([_.strip() for _ in address_list])

In [8]:
def save_df(df, output_path):
    try:
        df.to_csv(output_path, sep='\t', encoding='utf-8', index=False, header=True)
    except Exception as e:
         raise Exception('Could not save data to {0}: {1}'.format(output_path, e))


<br>
<br>

## 1. Querying current account balance for each address

<br>
<br>

#### 💡 This is a global variable which will be used all accross this notebook:

In [9]:
ADDRESS_LIST_DATA = open_file(ADDRESS_LIST)

In [10]:
def get_balance_addresses():
    
    balance_df = pd.DataFrame()
    addresses = get_addresses_str(ADDRESS_LIST_DATA)
    
    query = '?module=account&action=balancemulti&address={0}&tag=latest&apikey={1}'.format(addresses, ETHERSCAN_API)
    response = send_request(query)
    
    if handle_KeyError(response, 'message') == 'OK':
        
        for item in handle_KeyError(response, 'result'):   
            
            account = handle_KeyError(item,'account')
            balance = wei_to_eth(int(handle_KeyError(item,'balance')))
            
            balance_df = balance_df.append({'account': account, 'balance (ETH)': balance}, ignore_index=True)
            print('💰 Account {0} has {1} ETH\n'.format(account, balance))
            
        return balance_df
    
    else:
        print('🚨 Could not retrieve balance for {}'.format(account))

In [11]:
account_balance = get_balance_addresses()

account_balance

💰 Account 0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402 has 3.836709610517502 ETH

💰 Account 0x486d582eed105cef4e4aa270c93b1e03fe5b04f3 has 0.9389552956046704 ETH

💰 Account 0x0000000000000000000000000000000000000000 has 11359.137243443307 ETH

💰 Account 0xad05b50b71d1c05e3309e9f99e633a21741b77d9 has 0.11111116078411266 ETH

💰 Account 0x5a64f54ca8af1eafbcb31fec17042fc05c10aa14 has 0.111729086716969 ETH

💰 Account 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF has 0.3484970361548874 ETH

💰 Account FGTRY45YTHGFBGFHG has 0.0 ETH



,account,balance (ETH)
0,0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402,3.836710
1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,0.938955
2,0x0000000000000000000000000000000000000000,11359.137243
3,0xad05b50b71d1c05e3309e9f99e633a21741b77d9,0.111111
4,0x5a64f54ca8af1eafbcb31fec17042fc05c10aa14,0.111729
5,0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,0.348497
6,FGTRY45YTHGFBGFHG,0.000000


#### 💡 We will use account_balance to add a column with total balance to our next dataframes below.

<br>
<br>

### Saving dataframe to disk

In [12]:
output_path = os.path.join(OUTPUT_DIR, 'account_balance.csv')
save_df(account_balance, output_path)

<br>
<br>
<br>

## 2. Querying token transfers and swaps history
<br>
<br>

### ERC-20 tokens

In [13]:
def parse_erc20_list(response):
    
    result = pd.DataFrame(handle_KeyError(response, 'result'))
    
    result["timeStamp"] = pd.to_datetime(handle_KeyError(result, 'timeStamp'), unit='s')    
    result["value"] = wei_to_eth(handle_KeyError(result, 'value').apply(np.int))

    del result["blockNumber"]
    del result["hash"]
    del result["nonce"]
    del result["blockHash"]
    del result["tokenDecimal"]
    del result["transactionIndex"]
    del result["cumulativeGasUsed"]
    del result["input"]
    del result["confirmations"]
    del result["gas"]
    del result["gasPrice"]
    del result["gasUsed"]
    
    return result

In [14]:
def get_erc20_transfer_data(account):
    
    query = '?module=account&action=tokentx&address={0}&sort=asc&apikey={1}'.format(account.strip(), ETHERSCAN_API)
    response = send_request(query)
    
    if handle_KeyError(response, 'message') == 'OK':
        return parse_erc20_list(response)
    
    else:
        print('ERROR for{0}: {1}'.format(account, handle_KeyError(response, 'result')))
        return None

#### Let's check how the ERC20 transfer/swap data looks in the dataframe for one test account

In [15]:
account = '0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402'

get_erc20_transfer_data(account)

,timeStamp,from,contractAddress,to,value,tokenName,tokenSymbol
0,2021-04-10 22:46:43,0xe060724dccc684959dc31ffdcdaff2cc600281a8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0.050000,Wrapped Ether,WETH
1,2021-04-10 22:46:43,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x5b3256965e7c3cf26e11fcaf296dfc8807c01073,0.001250,Wrapped Ether,WETH
2,2021-06-02 20:16:01,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.081300,Wrapped Ether,WETH
3,2021-06-02 20:16:01,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x6b175474e89094c44da98b954eedeac495271d0f,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,221.654593,Dai Stablecoin,DAI
4,2021-06-02 20:17:40,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0x6b175474e89094c44da98b954eedeac495271d0f,0x4120c8c41656bae68d7e79872641fc8d716c3d42,221.654593,Dai Stablecoin,DAI
5,2021-07-09 23:16:50,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.320000,Wrapped Ether,WETH
6,2021-07-10 03:17:44,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.010000,Wrapped Ether,WETH
7,2021-07-10 03:19:27,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.010000,Wrapped Ether,WETH
8,2021-08-02 23:00:35,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.290000,Wrapped Ether,WETH
9,2021-08-10 21:41:25,0x5abfbe56553a5d794330eaccf556ca1d2a55647c,0x795065dcc9f64b5614c407a6efdc400da6221fb0,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0.368371,SushiSwap LP Token,SLP


<br>
<br>

### ERC-721 tokens

In [16]:
def parse_nft_list(response):

    result = pd.DataFrame(handle_KeyError(response, 'result'))
    result['timeStamp'] = pd.to_datetime(handle_KeyError(result, 'timeStamp'), unit='s') 

    del result["blockNumber"]
    del result["hash"]
    del result["nonce"]
    del result["blockHash"]
    del result["tokenDecimal"]
    del result["transactionIndex"]
    del result["cumulativeGasUsed"]
    del result["input"]
    del result["confirmations"]
    del result["gas"]
    del result["gasPrice"]
    del result["gasUsed"]
    
    return result

In [17]:
def get_erc721_transfer_data(account):
    
    query = '?module=account&action=tokennfttx&address={0}&apikey={1}'.format(account, ETHERSCAN_API)
    response = send_request(query)
    
    if handle_KeyError(response, 'message') == 'OK':
        return parse_nft_list(response)

    else:
        print('ERROR for{0}: {1}'.format(account, handle_KeyError(response, 'result')))
        return None

#### Let's check how the ERC721 transfer/swap data looks in the dataframe for one test account

In [18]:
account = '0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402'

get_erc721_transfer_data(account)

,timeStamp,from,contractAddress,to,tokenID,tokenName,tokenSymbol
0,2021-03-04 08:28:07,0x0000000000000000000000000000000000000000,0x60f80121c31a0d46b5279700f9df786054aa5ee5,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,256163,Rarible,RARI
1,2021-03-20 10:05:42,0x0000000000000000000000000000000000000000,0x60f80121c31a0d46b5279700f9df786054aa5ee5,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,539811,Rarible,RARI
2,2021-03-21 01:49:30,0x0000000000000000000000000000000000000000,0x60f80121c31a0d46b5279700f9df786054aa5ee5,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,550421,Rarible,RARI
3,2021-03-21 01:57:56,0x0000000000000000000000000000000000000000,0x60f80121c31a0d46b5279700f9df786054aa5ee5,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,550473,Rarible,RARI
4,2021-07-09 23:30:53,0x52e9c1e1f6d15700ee602ef72519d995df127ec9,0xc3f733ca98e0dad0386979eb96fb1722a1a05e69,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,24215,Acclimated​MoonCats,????
5,2021-07-10 03:14:53,0xa72bc016be8f075fdf24964fd62c422101574bb4,0x6f0365ca2c1dd63473f898a60f878a07e0f68a26,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,1326,TopDogBeachClub,TDBC
6,2021-07-10 03:20:31,0xf0aaf3c768a884fbbe0986fe95c44d10ec1f0b54,0xf78296dfcf01a2612c2c847f68ad925801eeed80,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,9682,FuckingPickles,FUCKINGPICKLES
7,2021-07-28 18:51:51,0xf76af70bc1ea5f49fe5583ab80493e116d602a34,0xe785e82358879f061bc3dcac6f0444462d4b5330,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,5801,World Of Women,WOW
8,2021-08-01 15:19:12,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc3f733ca98e0dad0386979eb96fb1722a1a05e69,0xeeaf86e05a95261290a871dd8cdb9470d5d3c9b7,24215,Acclimated​MoonCats,????
9,2021-08-05 01:55:44,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xe785e82358879f061bc3dcac6f0444462d4b5330,0xfc840af78a496b52e378dfa94838402e1b7f71dd,5801,World Of Women,WOW


<br>
<br>

### Looping over all the accounts and saving everything to a dataframe

In [19]:
def save_transfers_data_to_df():
    
    accounts = open_file(ADDRESS_LIST)
    
    list_keys = ['account', 'erc-20 transfers', 'erc-721 transfers']
    transfer_df = pd.DataFrame({list_keys[i]: [] for i in range(len(list_keys))})
    
    for account in accounts:
        
        account = account.strip()
        erc20_transfer_data = get_erc20_transfer_data(account) 
        erc721_transfer_data = get_erc20_transfer_data(account) 
        
        if type(erc20_transfer_data) != type(pd.DataFrame()):
            erc20_transfer_data = None
            
        if type(erc721_transfer_data) != type(pd.DataFrame()):
            erc721_transfer_data = None
        
        transfer_df = transfer_df.append(
            {'account': account, 
             'erc-20 transfers': erc20_transfer_data,
             'erc-721 transfers': erc721_transfer_data}, 
            ignore_index=True)
    
    return transfer_df

#### 💡 Note that will use this dataframe in the next section (finding tokens balance).

In [20]:
transfer_df = save_transfers_data_to_df()

transfer_df

ERROR forFGTRY45YTHGFBGFHG: Error! Invalid address format
ERROR forFGTRY45YTHGFBGFHG: Error! Invalid address format


,account,erc-20 transfers,erc-721 transfers
0,0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402,timeStamp ...,timeStamp ...
1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,timeStamp ...,timeStamp ...
2,0x0000000000000000000000000000000000000000,timeStamp ...,timeStamp ...
3,0xad05b50b71d1c05e3309e9f99e633a21741b77d9,timeStamp ...,timeStamp ...
4,0x5a64f54ca8af1eafbcb31fec17042fc05c10aa14,timeStamp ...,timeStamp ...
5,0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,timeStamp ...,timeStamp ...
6,FGTRY45YTHGFBGFHG,None,None


#### 💡 To access any account transfer data, simply grab its column number:

In [21]:
transfer_df.at[1, 'erc-20 transfers']

,timeStamp,from,contractAddress,to,value,tokenName,tokenSymbol
0,2021-04-14 15:39:52,0xef784caf2d2001fb8fbb9678f9a0a1b83cd582dc,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-18,CRYPTOPUNKS,Ͼ
1,2021-09-14 19:19:17,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C
2,2021-10-09 13:59:27,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C
3,2021-10-10 16:40:56,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C
4,2021-10-13 20:53:42,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C
5,2021-10-16 11:35:44,0xcbdeb4ff6cb834a83bf438d26a2fb5e6f53319b7,0xcbdeb4ff6cb834a83bf438d26a2fb5e6f53319b7,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,PunkETH.io,NFT
6,2021-10-18 10:10:45,0xcbdeb4ff6cb834a83bf438d26a2fb5e6f53319b7,0xcbdeb4ff6cb834a83bf438d26a2fb5e6f53319b7,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,PunkETH.io,NFT
7,2021-10-18 23:00:41,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C
8,2021-10-21 17:47:11,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C
9,2021-10-26 00:03:57,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x434d5875534d19fc7105281cf61dbfd1c93c8cb1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,1.000000e-09,CharPunk.Com,C


In [22]:
transfer_df.at[0, 'erc-721 transfers']

,timeStamp,from,contractAddress,to,value,tokenName,tokenSymbol
0,2021-04-10 22:46:43,0xe060724dccc684959dc31ffdcdaff2cc600281a8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0.050000,Wrapped Ether,WETH
1,2021-04-10 22:46:43,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x5b3256965e7c3cf26e11fcaf296dfc8807c01073,0.001250,Wrapped Ether,WETH
2,2021-06-02 20:16:01,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.081300,Wrapped Ether,WETH
3,2021-06-02 20:16:01,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x6b175474e89094c44da98b954eedeac495271d0f,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,221.654593,Dai Stablecoin,DAI
4,2021-06-02 20:17:40,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0x6b175474e89094c44da98b954eedeac495271d0f,0x4120c8c41656bae68d7e79872641fc8d716c3d42,221.654593,Dai Stablecoin,DAI
5,2021-07-09 23:16:50,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.320000,Wrapped Ether,WETH
6,2021-07-10 03:17:44,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.010000,Wrapped Ether,WETH
7,2021-07-10 03:19:27,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.010000,Wrapped Ether,WETH
8,2021-08-02 23:00:35,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x74de5d4fcbf63e00296fd95d33236b9794016631,0.290000,Wrapped Ether,WETH
9,2021-08-10 21:41:25,0x5abfbe56553a5d794330eaccf556ca1d2a55647c,0x795065dcc9f64b5614c407a6efdc400da6221fb0,0x2ee8d80de1c389f1254e94bc44d2d1bc391ed402,0.368371,SushiSwap LP Token,SLP


<br>
<br>

### Saving dataframe to disk

In [23]:
output_path = os.path.join(OUTPUT_DIR, 'transfer_data.csv')
save_df(transfer_df, output_path)

<br>
<br>
<br>


## 3. Current token balance

<br>
<br>


For each account, we are interested in finding out the current ERC-20 token balance. 
For this goal, we will continue leveraging Etherscan API to retrieve token balances for each account through the token's contract.

In [24]:
def get_contract_set(account_df) -> set():
    return set(account_df['contractAddress'])

In [25]:
def get_tokens_balance(contracts):

    token_balance_df = pd.DataFrame()   
    
    for contract in contracts:
        
        query = '?module=account&action=tokenbalance&contractaddress={0}&address={1}&tag=latest&apikey={2}'.format(contract, account, ETHERSCAN_API)
        response = send_request(query, less_log=True)

        if handle_KeyError(response, 'message') == 'OK'and int(handle_KeyError(response, 'result')) != 0.0:
            balance = wei_to_eth(int(handle_KeyError(response, 'result')))
            token_balance_df = token_balance_df.append(
                    {
                        'contract': contract,
                        'balance': balance
                    }, ignore_index=True)

    return token_balance_df


In [26]:
def get_account_contracts(transfer_df):
    
    tokens_balance_df = pd.DataFrame() 
    
    for item in range(len(ADDRESS_LIST_DATA)):

        account = transfer_df.at[item, 'account']
        contracts_list = transfer_df.at[item, 'erc-721 transfers']
        
        if type(contracts_list) != type(pd.DataFrame()):
            balances_df = None
        else:
            balances_df = get_tokens_balance(get_contract_set(contracts_list))
        
        tokens_balance_df = tokens_balance_df.append(
            {
                'account': account,
                'tokens balance': balances_df
            }, ignore_index=True)
    
    return tokens_balance_df

In [27]:
tokens_balance_df = get_account_contracts(transfer_df)

tokens_balance_df

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xc69e8504fdfbe3fa8857b4928331846f3760ffcc&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x7e9d8f07a64e363e97a648904a89fb4cd5fb94cd&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x6b175474e89094c44da98b954eedeac495271d0f&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x9d701e64896d99e215e1c621ac76a141fc403ca6&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x62491b0a7a4d0ddd31e9dcd65af939bf7167631f&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x274fd8346e35fc600f56b58992ed572350799a21&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xabdf2f974dac3305879ef50a43fb61f0ee5853f7&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xdb35a698114677a64dc479a258856403c2a8b490&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xdcc03ff18febdee3b12cdbd995eee7b7cba82fba&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x28f9eece184ddf04f93c6bd8977924f7f62308bd&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x50c8bfac03f030655632ff4ce1edf60264852af6&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x19e5d4e6034137ecadd73f06273473eb2a54e045&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xccbe8e7dde1a54e1d542117b4639dbca05d3cd00&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x8595a475d1c9a9d1468aad3a8809616c92998bea&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x729017c3e2cd949293f9b87197a8e84199ece769&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x9df2a2296c378a7a56dcda1bc18906045b299673&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xa1e83bce056237b54248c2e629eb064067270cf0&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x6d4de8732d6d059decaabea0e792a28668d5dba4&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xd253bd5c53d343a912281f2691902acced82963f&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

...

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xf4c408835de8c68232f4746b5ed598608b17e98d&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xe431d4b0a21ce82eb6f61f90eff5901f98cc7411&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xdc347e03d959df6ef767a37286160a11f963f551&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xe69ba3f5ffe577bba3a2bf8ce949f7875d4c67db&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x480f6ee5aec396d06da13180aad3f9af905f0705&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x138051fd239c5798d6ddc7157196506695379d0a&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xf87a5c644e2d8414e68ad0db99a3c63b5272528b&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xcccc13acce21916cf15d00ffa5587749b058135a&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xca06e159f73991ab58a424dde1b7f4fc056688e3&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x8dc46f303bf29429058466d3fdd21908cc28a5ce&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x76aba2986840d75527972c90ea1356f361ceea17&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x77dd6e7077d29ac95a8efbb5a4b2377809264cdc&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x213a6619dc06b6589f46c10551ad399650ed1784&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x3052704adba433417ba33e53daa2836f48efee8c&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0xc67dfc92e7bc5cedf15a41d290ce76548b71f003&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending request to: https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress=0x9744bdde8dd5c6441f797ec9e04fb337ff41d8d9&address=0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402&tag=latest&apikey=A597SW8N2GJUNNTQXEYNEVAC8KCNBW51QD
Sending requ

,account,tokens balance
0,0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402,balance ...
1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,balance ...
2,0x0000000000000000000000000000000000000000,Empty DataFrame Columns: [] Index: []
3,0xad05b50b71d1c05e3309e9f99e633a21741b77d9,balance ...
4,0x5a64f54ca8af1eafbcb31fec17042fc05c10aa14,balance ...
5,0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,balance ...
6,FGTRY45YTHGFBGFHG,None


#### 💡 At any time, we can retrieve an account tokens balance info with:

In [28]:
tokens_balance_df.at[0, 'tokens balance']

,balance,contract
0,50.000000,0xc69e8504fdfbe3fa8857b4928331846f3760ffcc
1,1.000000,0x7e9d8f07a64e363e97a648904a89fb4cd5fb94cd
2,0.195074,0x6b175474e89094c44da98b954eedeac495271d0f
3,0.009464,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
4,500.000000,0x90275c752ff8e020cdeec4dec43318cf569fd445
5,10.549045,0x58b6a8a3302369daec383334672404ee733ab239
6,300.000000,0x9500b696f00ae82ca97d06379cf0a2b60b467040
7,2.407900,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2
8,5.820733,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
9,55.000000,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0


### Cleaning up the data

The data above is nice, but it's hard to know what token each contract represents. Because Etherescan does not offer a contract lookup for free accounts, we will leverage the [web3.py](https://web3py.readthedocs.io/en/stable/) library to complete our data with this info.

Let's create a global instance of this provider that can be used for all the queries in the methods below.

In [29]:
w3 = Web3(Web3.HTTPProvider(INFURA_URL))
w3.isConnected()

True

In [30]:
def get_token_symbol(contract) -> str:
    contract = Web3.toChecksumAddress(contract)
    minimal_abi = [{"inputs":[],"name": "symbol","outputs":[{"name": "","type": "string"}],"type": "function"},]
    contract = w3.eth.contract(contract, abi=minimal_abi)
    return contract.functions.symbol().call()

In [31]:
def format_float(num) -> float:
    FLOAT_PRECISION = '.6f'
    return format(float(num), FLOAT_PRECISION)

In [32]:
def cleaning_tokens_balance_dt(tokens_balance_df):
    
    new_tokens_balance_df = pd.DataFrame()
    
    for row in range(len(ADDRESS_LIST_DATA)):
        
        account = tokens_balance_df.at[row, 'account']
        contracts_df = tokens_balance_df.at[row, 'tokens balance']
        
        if type(contracts_df) != type(pd.DataFrame()) or contracts_df.empty:
            continue
        
        for i, contract in enumerate(contracts_df['contract']):
            
            balance = format_float(contracts_df.at[i, 'balance'])
            token_symbol = get_token_symbol(contract)
          
            new_tokens_balance_df = new_tokens_balance_df.append({
                'contract': contract,
                'balance':  balance,
                'symbol': token_symbol
            }, ignore_index=True)
    
        tokens_balance_df.at[row, 'tokens balance'] = new_tokens_balance_df
        
    tokens_balance_df['balance (ETH)'] = account_balance['balance (ETH)']

    return tokens_balance_df

In [33]:
cleaning_tokens_balance_dt(tokens_balance_df)

,account,tokens balance,balance (ETH)
0,0x2ee8D80de1c389f1254e94bc44D2d1Bc391eD402,balance ...,3.836710
1,0x486d582eed105cef4e4aa270c93b1e03fe5b04f3,balance ...,0.938955
2,0x0000000000000000000000000000000000000000,Empty DataFrame Columns: [] Index: [],11359.137243
3,0xad05b50b71d1c05e3309e9f99e633a21741b77d9,balance ...,0.111111
4,0x5a64f54ca8af1eafbcb31fec17042fc05c10aa14,balance ...,0.111729
5,0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,balance ...,0.348497
6,FGTRY45YTHGFBGFHG,None,0.000000


#### 💡 To access a particular account, simply use its column number:

In [34]:
tokens_balance_df.at[1, 'tokens balance']

,balance,contract,symbol
0,50.000000,0xc69e8504fdfbe3fa8857b4928331846f3760ffcc,REEL
1,1.000000,0x7e9d8f07a64e363e97a648904a89fb4cd5fb94cd,FF
2,0.195074,0x6b175474e89094c44da98b954eedeac495271d0f,DAI
3,0.009464,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,gOHM
4,500.000000,0x90275c752ff8e020cdeec4dec43318cf569fd445,JENIL
5,10.549045,0x58b6a8a3302369daec383334672404ee733ab239,LPT
6,300.000000,0x9500b696f00ae82ca97d06379cf0a2b60b467040,LFG
7,2.407900,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,SUSHI
8,5.820733,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI
9,55.000000,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0,MATIC


<br>
<br>

### Saving dataframe to disk

In [35]:
output_path = os.path.join(OUTPUT_DIR, 'tokens_balance.csv')
save_df(tokens_balance_df, output_path)